Hi! I'm kaggle newbi!

I use catalyst, smp, and albumentations.
They run on **pytorch** and are very convinience tools!

Segmentation_models(smp) is High level API for image segmentation.
https://github.com/qubvel/segmentation_models.pytorch

If you want to use other models or pretrained encoder, you just do type this.
```
ENCODER = 'efficientnet-b3'
```

Albumentations is a Python library for image augmentation.
https://github.com/albumentations-team/albumentations

If you want to augmentate images, you just do type this!
```
albu.Transpose(p=0.5), 
albu.RandomRotate90(3)
```

This notebook create on google colab. So If you load all data to google drive, you can run this notebook on colab!
(Is there any method what I up load notebook with printed shell? I want to display result of this with code.)


I challenge to submit test data, but my kaggle kernel shutdown when I convert mask to encoding.
haha..  :0

Any comments are welcome! I really appreciate your opinion.


# 1. Install library

In [ ]:
from IPython.display import clear_output
!pip install catalyst
!pip install segmentation_models_pytorch
!pip install albumentations==0.3.2
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir # for albu.lambda
clear_output()

In [ ]:
from PIL import Image
import tifffile as tiff
import subprocess
import pandas as pd
from IPython.display import clear_output
import matplotlib.pyplot as plt

import numpy as np
import cv2
import os
from tqdm.notebook import tqdm
import zipfile
from sklearn.model_selection import train_test_split


import torchvision
import torchvision.transforms as transforms
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import albumentations as albu
#from albumentations import torch as AT
from albumentations import Compose,Resize,OneOf,RandomBrightness,RandomContrast,Normalize,HorizontalFlip,Blur,ElasticTransform,GridDistortion,OpticalDistortion,GaussNoise 
from albumentations.pytorch import ToTensor
from catalyst.data import Augmentor
from catalyst.dl import utils
from catalyst.data.reader import ScalarReader, ReaderCompose, LambdaReader#ImageReader
from catalyst.dl.runner import SupervisedRunner
#from catalyst.contrib.models.segmentation import Unet
from catalyst.dl.callbacks import DiceCallback, EarlyStoppingCallback, InferCallback, CheckpointCallback
from catalyst.dl.callbacks import JaccardCallback,PrecisionRecallF1ScoreCallback
import segmentation_models_pytorch as smp


seed = 1015
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# 2. Create 256 * 256 images
(Thanks Iafoss for your share!)

I save files on google drive, run this part only one time.**

In [ ]:
train_meta = pd.read_csv('/content/drive/MyDrive/kaggledrive/kidney/data/train.csv').set_index('id')

In [ ]:
# load original data to google drive
# load it to colab

train_path = "/content/drive/MyDrive/kaggledrive/kidney/data/train_img/"
for idx in train_meta.index:
    subprocess.call(["cp",
                     f"{train_path}{idx}.json",
                     f"{idx}.json"] ,shell = False)
    subprocess.call(["cp",
                     f"{train_path}{idx}-anatomical-structure.json",
                     f"{idx}-anatomical-structure.json"] ,shell = False)
    subprocess.call(["cp",
                    f"{train_path}{idx}.tiff",
                    f"{idx}.tiff"] ,shell = False)

In [ ]:
#functions to convert encoding to mask and mask to encoding
def enc2mask(encs, shape):
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for m,enc in enumerate(encs):
        if isinstance(enc,np.float) and np.isnan(enc): continue
        s = enc.split()
        for i in range(len(s)//2):
            start = int(s[2*i]) - 1
            length = int(s[2*i+1])
            img[start:start+length] = 1 + m
    return img.reshape(shape).T

def mask2enc(mask, shape, n=1):
    pixels = mask.T.flatten()
    encs = []
    for i in range(1,n+1):
        p = (pixels == i).astype(np.int8)
        if p.sum() == 0: encs.append(np.nan)
        else:
            p = np.concatenate([[0], p, [0]])
            runs = np.where(p[1:] != p[:-1])[0] + 1
            runs[1::2] -= runs[::2]
            encs.append(' '.join(str(x) for x in runs))
    return encs

In [ ]:
DATA = ""
sz = 256   #the size of tiles
reduce = 4 #reduce the original images by 4 times 
OUT_TRAIN = 'train_object.zip'
OUT_MASKS = 'masks_object.zip'

s_th = 40  #saturation blancking threshold
p_th = 200*sz//256 #threshold for the minimum number of pixels

x_tot,x2_tot = [],[]
with zipfile.ZipFile(OUT_TRAIN, 'w') as img_out, zipfile.ZipFile(OUT_MASKS, 'w') as mask_out:
    for index, encs in tqdm(train_meta.iterrows(),total=len(train_meta)):
        #read image and generate the mask
        img = tiff.imread(os.path.join(DATA,index+'.tiff'))
        if len(img.shape) == 5:img = np.transpose(img.squeeze(), (1,2,0))
        mask = enc2mask(encs,(img.shape[1],img.shape[0]))

        #add padding to make the image dividable into tiles
        shape = img.shape
        pad0,pad1 = (reduce*sz - shape[0]%(reduce*sz))%(reduce*sz), (reduce*sz - shape[1]%(reduce*sz))%(reduce*sz)
        img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],constant_values=0)
        mask = np.pad(mask,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2]],constant_values=0)

        #split image and mask into tiles using the reshape+transpose trick
        img = cv2.resize(img,(img.shape[0]//reduce,img.shape[1]//reduce),interpolation = cv2.INTER_AREA)
        
        img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
        img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)

        mask = cv2.resize(mask,(mask.shape[0]//reduce,mask.shape[1]//reduce),interpolation = cv2.INTER_NEAREST)
        mask = mask.reshape(mask.shape[0]//sz,sz,mask.shape[1]//sz,sz)
        mask = mask.transpose(0,2,1,3).reshape(-1,sz,sz)

        #write data
        for i,(im,m) in enumerate(zip(img,mask)):
            #remove black or gray images based on saturation check
            hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
            h, s, v = cv2.split(hsv)
            if (s>s_th).sum() <= p_th or im.sum() <= p_th: continue
            if m.sum() <= 1 : continue
    
            x_tot.append((im/255.0).reshape(-1,3).mean(0))
            x2_tot.append(((im/255.0)**2).reshape(-1,3).mean(0))
            
            im = cv2.imencode('.png',cv2.cvtColor(im, cv2.COLOR_RGB2BGR))[1]
            img_out.writestr(f'{index}_{i}.png', im)

            m = cv2.imencode('.png',m)[1]
            mask_out.writestr(f'{index}_{i}.png', m)

#image stats
img_avr =  np.array(x_tot).mean(0)
img_std =  np.sqrt(np.array(x2_tot).mean(0) - img_avr**2)
print('mean:',img_avr, ', std:', img_std)


In [ ]:
# save data to google drive

path = "/content/drive/MyDrive/kaggledrive/kidney/data/"
!cp train_object.zip "{path}train_object_256.zip"
!cp masks_object.zip "{path}masks_object_256.zip"

# 3. Load data

In [ ]:
train_meta = pd.read_csv('/content/drive/MyDrive/kaggledrive/kidney/data/train.csv').set_index('id')

In [ ]:
# make directory on colab
!mkdir /content/train_object_256
!mkdir /content/masks_object_256
# make logs directory on colab
!mkdir logs 

# load 256 256 data
path = "/content/drive/MyDrive/kaggledrive/kidney/data/"
!cp "{path}train_object_256.zip" "/content/train_object_256/train_object.zip"
!cp "{path}masks_object_256.zip" "/content/masks_object_256/masks_object.zip "

!unzip "/content/train_object_256/train_object.zip"  -d "/content/train_object_256"
!unzip "/content/masks_object_256/masks_object.zip " -d "/content/masks_object_256"

clear_output()

# 4. Create Dataset

In [ ]:
namelist = []
with zipfile.ZipFile("/content/train_object_256/train_object.zip", 'r') as img_arch:
    namelist = img_arch.namelist().copy()

In [ ]:
class Kidney_Dataset(torch.utils.data.Dataset):
    def __init__(self, namelist, 
                 transform=None,
                 preprocessing=None,
                 classes=1, 
                 augmentation=None, 
                ):
        self.namelist = namelist
        self.transforms = transform
        self.classes = classes
        self.preprocessing = preprocessing
        self.augmentation = augmentation
        self.imgsize = 256
        '''
        self.resize =  Compose([
                                  albu.Resize(height = self.imgsize, width = self.imgsize),
                               ])
        '''
        self.to_tensor = Compose([
                                  albu.Lambda(image= to_tensor, mask=to_tensor),
                               ])
        
        
    def __getitem__(self,index):
        img_name = self.namelist[index]
        # Read Data----------------------------------------
        img = np.array(Image.open("train_object_256/"+img_name))/256
        mask = np.array(Image.open("masks_object_256/"+img_name))[...,np.newaxis]
        
        # Using background channel is usefull? ...> need to experient
        
        #background  = np.ones(mask.shape)
        #background = background - mask
        #mask = np.concatenate([mask,background], axis = 2)

        #apply augmentation
        if self.augmentation:
            sample = self.augmentation(image=img, mask=mask)
            img, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=img, mask=mask)
            img, mask = sample['image'], sample['mask']

        # reshape for converting to tensor
        sample = self.to_tensor(image=img, mask=mask)
        img, mask = sample['image'], sample['mask']
        
        return img, mask
    
    def __len__(self):
        return len(self.namelist)

In [ ]:
def get_training_augmentation():
    transform = [
        albu.Transpose(p=0.5),   # * 2
        albu.RandomRotate90(3),  # * 4
        #albu.Rotate(p=1),

        albu.ShiftScaleRotate(p = 1),
        albu.RandomSizedCrop(min_max_height=(196, 256), height = 256, width = 256, p = 1),#(128,256)
        albu.GridDistortion(p = 0.5), # * 2

        #albu.GridDropout(ratio= 0.4, mask_fill_value = 0, p = 1)
        #albu.GridDropout(ratio= 0.4, mask_fill_value = None, p = 0.5, random_offset = True)

        ]
    return albu.Compose(transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

def get_preprocessing(preprocessing_fn):
    _transform = [
        albu.Lambda(image = preprocessing_fn),
    ]
    return albu.Compose(_transform)

In [ ]:
dataset = Kidney_Dataset(namelist, augmentation = get_training_augmentation(),  classes=1)

In [ ]:
img_id = 17
plt.figure(figsize=(10,10))
img, mask = dataset[img_id]
plt.imshow(img.transpose([1,2,0])) 
plt.imshow(mask[0,:,:], alpha=0.3) 
plt.show()

# 5. Create model

In [ ]:
ENCODER = 'efficientnet-b3'
ENCODER_WEIGHTS = 'imagenet'
DEVICE = 'cuda'

ACTIVATION = 'sigmoid'

model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    in_channels = 3,
    classes=1, 
    activation = ACTIVATION
)


In [ ]:
train_names, test_names = train_test_split(namelist, test_size=0.10, random_state=1015)

#preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)
train_dataset = Kidney_Dataset(
    train_names,
    augmentation=get_training_augmentation(), 
    preprocessing=None, #get_preprocessing(preprocessing_fn),
    classes=1,
)

valid_dataset = Kidney_Dataset(
    test_names,
    augmentation = None,
    preprocessing= None, #get_preprocessing(preprocessing_fn),
    classes=1,
)

BATCH_SIZE = 32

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, persistent_workers = True)
valid_loader = DataLoader(valid_dataset,  batch_size=40, shuffle=False, num_workers=2)
loaders = {
    "train": train_loader,
    "valid": valid_loader
}

In [ ]:
num_epochs = 100
optimizer = torch.optim.Adam([
    {'params': model.decoder.parameters(), 'lr': 1e-2}, 
    {'params': model.encoder.parameters(), 'lr': 1e-3},  
])

scheduler = ReduceLROnPlateau(optimizer, factor=0.15, patience=4)
criterion = smp.utils.losses.DiceLoss(eps=1.)
runner = SupervisedRunner(device=device)


# 6. Train

In [ ]:
logdir = "./logs"
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    callbacks=[DiceCallback(threshold = 0.5),
               JaccardCallback(),
               EarlyStoppingCallback(patience=10, min_delta=0.001),
               ],
    logdir=logdir,
    num_epochs=num_epochs,
    verbose=True
)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir {'logs'}

In [ ]:
# save model on google drive

PATH = "/content/drive/MyDrive/kaggledrive/kidney/model/"
#torch.save(runner.model, PATH +"efficientnet_b3.pt")
torch.save(runner.model, "efficientnet_b3.pt")

# 7. Visualization of model perpomence

In [ ]:
# load model

PATH = "/content/drive/MyDrive/kaggledrive/kidney/model/"
model_new = torch.load(PATH +"efficientnet_b3.pt")
model_new.eval()

runner = SupervisedRunner(device=device)
runner.model = model_new

## Train_Set

In [ ]:
num = 3
for i, train_batch in enumerate(loaders['train']):
    plt.figure(figsize = (12,4))
    train_sample, train_mask = train_batch
    train_out = runner.predict_batch({"features": train_sample.cuda()})['logits']
    plt.subplot(1,3,1)
    
    plt.imshow(train_sample[num].permute(1,2,0).numpy()[:,:,0],cmap='bone') 
    plt.gca().set_title("Original")

       
    plt.subplot(1,3,2)
    plt.imshow(train_sample[num].permute(1,2,0).numpy()[:,:,0], cmap='bone') 
    plt.imshow(train_mask[num,0,:,:].cpu().numpy(),alpha=0.3) 
    plt.gca().set_title("Real Tissue")

    plt.subplot(1,3,3)
    plt.imshow(train_sample[num].permute(1,2,0).numpy()[:,:,0], cmap='bone')
    plt.imshow(train_out[num,0,:,:].cpu().numpy(),alpha = 0.3)
    plt.gca().set_title("Predicted Tissue")
    
    plt.show()
    if i >10:
        break

# validation set

In [ ]:
for i, test_batch in enumerate(loaders['valid']):
    test_sample, test_mask = test_batch
    test_out = runner.predict_batch({"features": test_sample.cuda()})['logits']
    for j in range(len(test_out)):
        plt.figure(figsize = (12,4))
        plt.subplot(1,3,1)
        
        plt.imshow(test_sample[j].permute(1,2,0).numpy()[:,:,0], cmap='bone') 
        plt.gca().set_title("Original")
        
        plt.subplot(1,3,2)
        plt.imshow(test_sample[j].permute(1,2,0).numpy()[:,:,0], cmap='bone')
        plt.imshow(test_mask[j,0,:,:].cpu().numpy(), alpha = 0.3) 
        plt.gca().set_title("Real Tissue")

        
        plt.subplot(1,3,3)
        plt.imshow(test_sample[j].permute(1,2,0).numpy()[:,:,0],cmap='bone')
        plt.imshow(test_out[j,0,:,:].cpu().numpy(),alpha = 0.3)
        plt.gca().set_title("Predicted Tissue")
        plt.show()

plt.show()